# Анализ эффективности удержания

In [74]:
import warnings
warnings.filterwarnings('ignore')
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


В этом задании вам предлагается проанализировать данные одной из американских телекоммуникационных компаний о пользователях, которые потенциально могут уйти.

Измерены следующие признаки:

state — штат США

account_length — длительность использования аккаунта

area_code — деление пользователей на псевдорегионы, использующееся в телекоме

intl_plan — подключена ли у пользователя услуга международного общения

vmail_plan — подключена ли у пользователя услуга голосовых сообщений

vmail_message — количество голосых сообщений, который пользователь отправил / принял

day_calls — сколько пользователь совершил дневных звонков

day_mins — сколько пользователь проговорил минут в течение дня

day_charge — сколько пользователь заплатил за свою дневную активность

eve_calls, eve_mins, eve_charge — аналогичные метрики относительно вечерней активности

night_calls, night_mins, night_charge — аналогичные метрики относительно ночной активности

intl_calls, intl_mins, intl_charge — аналогичные метрики относительно международного общения

custserv_calls — сколько раз пользователь позвонил в службу поддержки

treatment — номер стратегии, которая применялись для удержания абонентов (0, 2 = два разных типа воздействия, 1 = контрольная группа)

mes_estim — оценка интенсивности пользования интернет мессенджерами

churn — результат оттока: перестал ли абонент пользоваться услугами оператора


In [7]:
data = pd.read_csv('churn_analysis.csv', sep = ',',index_col = 0)

In [8]:
data.head()

,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
1,OH,107,415,no,yes,26,161.6,123,27.47,195.5,...,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False.
2,NJ,137,415,no,no,0,243.4,114,41.38,121.2,...,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False.
3,OH,84,408,yes,no,0,299.4,71,50.90,61.9,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
4,OK,75,415,yes,no,0,166.7,113,28.34,148.3,...,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False.


Давайте рассмотрим всех пользователей из контрольной группы (treatment = 1). Для таких пользователей мы хотим проверить гипотезу о том, что штат абонента не влияет на то, перестанет ли абонент пользоваться услугами оператора.

Для этого мы воспользуемся критерием хи-квадрат. Постройте таблицы сопряженности между каждой из всех 1275 возможных неупорядоченных пар штатов и значением признака churn. Для каждой такой таблицы 2x2 применить критерий хи-квадрат можно с помощью функции

scipy.stats.chi2_contingency(subtable, correction=False)

Заметьте, что, например, (AZ, HI) и (HI, AZ) — это одна и та же пара. Обязательно выставьте correction=False (о том, что это значит, вы узнаете из следующих вопросов).

Сколько достигаемых уровней значимости оказались меньше, чем α=0.05?

In [13]:
data_control = data.query('treatment == 1')
data_control.head(3)

,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
3,OH,84,408,yes,no,0,299.4,71,50.90,61.9,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
8,LA,117,408,no,no,0,184.5,97,31.37,351.6,...,215.8,90,9.71,8.7,4,2.35,1,1,0.50,False.


In [33]:
pivot = pd.pivot_table(data_control, index = 'state', columns = 'churn', aggfunc='size', fill_value=0)
pivot.head(2)

churn,False.,True.
state,,
AK,19,1
AL,25,5


In [48]:
from itertools import combinations
combo = list(combinations(pivot.index, 2))

In [93]:
p = []
for i in combo:
    temp_p = sc.stats.chi2_contingency(pivot.loc[[i[0], i[1]], :], correction=False)
    p.append(temp_p[1])
p = np.asarray(p)
len(p[p<0.05])

34

In [94]:
np.average(p)

0.5018273798739158

Проведите те же самые сравнения, что и в вопросе №1, только с включенной коррекцией

In [69]:
p_1 = []
for i in combo:
    temp_p = sc.stats.chi2_contingency(pivot.loc[[i[0], i[1]], :], correction=True)
    p_1.append(temp_p[1])
p_1 = np.asarray(p_1)
len(p_1[p_1<0.05])

0

In [91]:
np.average(p_1)

0.6640566382051047

Посчитайте для каждой пары штатов, как и в первом задании, достигаемый уровень значимости с помощью точного критерия Фишера и сравните получившиеся значения с двумя другими подходами, описанными выше.

In [99]:
p = []
import scipy.stats as stats
for i in combo:
    temp_p = stats.fisher_exact(pivot.loc[[i[0], i[1]], :])
    p.append(temp_p[1])
p = np.asarray(p)
len(p[p<0.05])

10

In [100]:
np.average(p)

0.6483383060020681

Давайте попробуем применить полученные знания о разных видах корреляции и ее применимости на практике.

Рассмотрим пару признаков day_calls и mes_estim. Посчитайте корреляцию Пирсона между этими признаками на всех данных, ее значимость.

Корреляция Пирсона:

In [83]:
val, p = stats.pearsonr(data['day_calls'],data['mes_estim'])

In [87]:
print(val, p)

-0.051794350587572625 0.0027798836869738384


Посчитайте корреляцию Спирмена между этими признаками на всех данных, ее значимость.

In [88]:
val, p = stats.spearmanr(data['day_calls'],data['mes_estim'])
print(val, p)

0.043349880533927444 0.012317367189170541
